# Full algorithm - generating grocery lists

1. Define variables: days-times-stores
2. Define limits: min amount-max amount
3. Generate (random) lists from narrowed down df

### Import libraries 

In [4]:
    # Function to generate #visits per day of the week
from AlgorithmDAYS_HH2 import beforeNY, AfterNY, daysAfterNY

    # Function to generate store name (&type) per weekday (+ assign #of visits per day)
from AlgorithmSTORE_HH2 import storeperday, countsperstore

    #Function to define the limits for total visits, days, same store and type per week
from AlgorithmCOUNTS_HH2 import CountTotalVisits, CountTotalDays, CountStoreName, CountStoreType, CountTotalPerday, CountVisitsPerDay, CountTypePerStore

    # Function to decide time (morning, noon. afternoon) per visited store
from AlgorithmTIMES_HH2 import AlbertHeijn, Okay, Delhaize, Carrefour, VersavelPoelman, Sys, Brabo, Ikea, Kruidvat

    # Function to decide amounts per visit & per week
from AlgorithmAMOUNT_HH2 import StoreAmount, WeekAmount, StoreCount, WeekCount, CatCount, PerCatCount, PerCatCountALL, PerCatCountSTORE

    # Function to make the grocery lists
from AlgorithmITEMS_HH2 import supermarket, bakery, butcher, drugstore, furniturestore, supermarkets, bakeries, butchers, drugstores, furnstores

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dataframe_image as dfi
from datetime import time
import matplotlib.dates as mdates
from matplotlib.ticker import StrMethodFormatter
from matplotlib.pyplot import figure
from random import sample

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)

class bcolors:
    WARNING = '\033[91m'
    BOLD = '\033[1m'

In [2]:
dataframe = pd.read_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2.csv")
df = pd.read_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2.csv")

# Define variables - day/time/store

## Days & visits

In [13]:
df1 = daysAfterNY(dataframe)
display(df1)

print(df1['times'].sum())

,times,day
4,2.0,Monday
5,1.0,Tuesday
4,0.0,Wednesday
4,0.0,Thursday
4,0.0,Friday
5,1.0,Saturday
4,0.0,Sunday


4.0


## Stores & time

In [51]:
df2 = storeperday(df1)
df2

 RERUN - Too many stores on: Tuesday


,store_name,store_type,day,time
0,Albert Heijn,supermarket,Monday,noon
1,Sys,bakery,Monday,morning
2,Brabo,butcher,Thursday,afternoon
3,Okay,supermarket,Saturday,afternoon


## Define limits per week (item counts)

In [42]:
WeekCount()

(9, 86)

In [58]:
df4 = countsperstore(df2)
df4['I'] = np.arange(df4.shape[0])
df4

,store_name,store_type,day,time,counts,I
0,Albert Heijn,supermarket,Monday,noon,1,0
1,Sys,bakery,Monday,morning,5,1
2,Brabo,butcher,Thursday,afternoon,6,2
3,Okay,supermarket,Saturday,afternoon,31,3


In [54]:
data = {'store_name': ['Albert Heijn', 'Sys', 'Okay', 'Versavel Poelman'],
       'store_type': ['supermarket', 'bakery', 'supermarket', 'butcher'],
        'day': ['Thursday', 'Saturday', 'Saturday', 'Saturday'],
       'time': ['noon', 'noon', 'afternoon', 'morning'],
       'counts': [3, 5, 35, 7],}
df4 = pd.DataFrame(data)
df4['I'] = np.arange(df4.shape[0])
df4

,store_name,store_type,day,time,counts,I
0,Albert Heijn,supermarket,Thursday,noon,3,0
1,Sys,bakery,Saturday,noon,5,1
2,Okay,supermarket,Saturday,afternoon,35,2
3,Versavel Poelman,butcher,Saturday,morning,7,3


# The Grocery Lists

In [11]:
CountTypePerStore('lunch salad', 'supermarket')

(0.0, 2.0)

In [12]:
CountTotalPeritemSTORE = CountTypePerStore('lunch salad', 'supermarket')
minitemperdaySTORE = CountTotalPeritemSTORE[0]
maxitemeperdaySTORE = CountTotalPeritemSTORE[1]
maxitemeperdaySTORE

2.0

In [66]:
restart = True 
while restart:
    # GET A LIST PER STORE VISITED IN THIS WEEK [ADJUST ACCORDING TO WEEKPLANNING]
    super1 = supermarkets(df4, 0)
    super2 = supermarkets(df4, 3)
    baker1 = bakeries(df4, 1)
    butcher1 = butchers(df4, 2)
    
    # get lists per store type
    supermarketlist = pd.concat([super1, super2])
    bakerlist = pd.concat([baker1])
    butcherlist = pd.concat([butcher1])

    # append the different lists to the entire week
    weeklist = [super1, super2, baker1, butcher1]
    week = pd.concat(weeklist)

    #########
    # Check if the categories per WEEK counts are ok:
    for category in df['category']:
            percatcount = week[week['category']==category]['category'].count()
            CountTotalPerCat = PerCatCountALL(category)
            mincatperday = CountTotalPerCat[0]
            maxcateperday = CountTotalPerCat[1]
            if mincatperday <= percatcount <= maxcateperday:
                pass
            else:
                print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN -  week - Too many/few of category:", category)
                restart = True
                break       
                
    for items in df['item_type']:
            week9Amount = week['amount'].sum()
            NormWeekAmount = WeekAmount()
            minWeekAmount = NormWeekAmount[0]
            maxWeekAmount = NormWeekAmount[1]
            if minWeekAmount <= week9Amount <= maxWeekAmount:
                pass
            else:
                print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - AMOUNT/WEEK - to many:", week9Amount)
                restart = True
                break  
               
    # Check if the itemtype counts PER STORE TYPE are ok:
    for itemtype in df['item_type']:
            peritemcountSTORE = supermarketlist[supermarketlist['item_type']==itemtype]['item_type'].count()
            CountTotalPeritemSTORE = CountTypePerStore(itemtype, 'supermarket')
            minitemperdaySTORE = CountTotalPeritemSTORE[0]
            maxitemeperdaySTORE = CountTotalPeritemSTORE[1]
            if minitemperdaySTORE <= peritemcountSTORE <= maxitemeperdaySTORE:
                restart = False
                continue
            else:
                print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN -  week - Too many/few of item:", itemtype)
                restart = True
                break
            print('gud')

## Extra Check

In [67]:
# Check if the itemtype counts PER STORE TYPE are ok:
for itemtype in df['item_type']:
        peritemcountSTORE = supermarketlist[supermarketlist['item_type']==itemtype]['item_type'].count()
        CountTotalPeritemSTORE = CountTypePerStore(itemtype, 'supermarket')
        minitemperdaySTORE = CountTotalPeritemSTORE[0]
        maxitemeperdaySTORE = CountTotalPeritemSTORE[1]
        if minitemperdaySTORE <= peritemcountSTORE <= maxitemeperdaySTORE:
            pass
        else:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN -  week - Too many/few of item:", itemtype)
            break
    

# Check if the categories per WEEK counts are ok:
for category in df['category']:
        percatcount = week[week['category']==category]['category'].count()
        CountTotalPerCat = PerCatCountALL(category)
        mincatperday = CountTotalPerCat[0]
        maxcateperday = CountTotalPerCat[1]
        if mincatperday <= percatcount <= maxcateperday:
            pass
        else:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN -  week - Too many/few of category:", category)
            break    
            
            
# Check if the total item amount per week is ok:
for items in df['item_type']:
    week9Amount = week['amount'].sum()
    NormWeekAmount = WeekAmount()
    minWeekAmount = NormWeekAmount[0]
    maxWeekAmount = NormWeekAmount[1]
    if week9Amount < minWeekAmount:
        print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - AMOUNT/WEEK - to few:", week9Amount)
        break
    elif maxWeekAmount < week9Amount:
        print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - AMOUNT/WEEK - to many:", week9Amount)
        break 
    else:
        continue
print('all = gud!')

all = gud!


# OUTPUT: Call the grocery lists

In [63]:
week.sort_values(by=['category']).head()

,item_type,item_name,store_type,store_name,amount,category,weights_itemtype
6,danish,CHOCOLADEKOEK,bakery,Sys,2,bakery,9
126,danish,CHOCOLADEKOEK,bakery,Sys,1,bakery,9
127,danish,VIERK. CREMEKOEK CH,bakery,Sys,1,bakery,9
296,danish,BOOTJE,bakery,Sys,2,bakery,9
39,pita bread,1X5 PITABROODJES,supermarket,Delhaize,1,bakery,2


In [57]:
df4

,store_name,store_type,day,time,counts,I
0,Albert Heijn,supermarket,Monday,noon,2,0
1,Versavel Poelman,butcher,Monday,morning,7,1
2,Okay,supermarket,Thursday,afternoon,34,2
3,Sys,bakery,Saturday,morning,5,3


In [68]:
# Check individual lists (if supermarket s[i], if bakery ba[i]; if butcher bu[i])

super2.sort_values(by=['category'])

,item_type,item_name,store_type,store_name,amount,category,weights_itemtype
48,tea,20 CLI RAS7MIN BIO,supermarket,Delhaize,1,beverages,6
24,coke,6X33C COCA ZERO,supermarket,Delhaize,1,beverages,4
59,apple sauce,EVERYDAY appelcompote stuk 710g,supermarket,Okay,1,canned foods,4
77,pasta sauce,SPAGHERONI pastasaus tradizionale 520g,supermarket,Okay,1,condiments,1
58,andalouse sauce,BONI saus andalouse TD 420ml,supermarket,Okay,1,condiments,1
68,coconut milk,BONI kokosmelk blik 400g,supermarket,Okay,2,cooking aid,2
19,cheese,WATOU SPECIAL,supermarket,Delhaize,1,dairy & plant based,20
64,cheese,BONI gouda light sneetjes 250g,supermarket,Okay,1,dairy & plant based,20
22,chocolate milk,1L CECEMEL LESS-SU,supermarket,Delhaize,1,dairy & plant based,2
334,cheese,MORA Oven goudse kaasballetjes 14x20g,supermarket,Okay,1,dairy & plant based,20


In [52]:
# Check individual lists (if supermarket s[i], if bakery ba[i]; if butcher bu[i])
baker

,item_type,item_name,store_type,store_name,amount,category,weights_itemtype
40,bread,KLEIN FRANS BROOD,bakery,Sys,2,bakery,16
170,danish,SUIS GELE CRÈME,bakery,Sys,1,bakery,9
198,fish spread,DIVERSEN: NOORDZEESALADE,bakery,Sys,1,"meat, fish & veggie",4
172,danish,CHOCOLADE CRÈME KOEK,bakery,Sys,4,bakery,9
308,sandwich,SANDWICH,bakery,Sys,4,bakery,7
